### CNN : convolution layer + pooling layer
**지정한 영역의 값들을 하나의 값으로 압축**
 - convolution layer : 가중치와 편향 적용 <br/>(윈도우 크기만큼의 가중치와 1개의 편향 ) = 커널
 - pooling layer : 값들 중 하나 선택
 - 윈도우 : 지정한 크기의 영역, 이 값을 움직이며 은닉층 완성
 - 스트라이드 : 윈도우를 몇 칸씩 움직일지에 대한 값

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

In [6]:
# first layer 

# convolution layer
# W1 [3, 3, 1, 32] -> [3 3]: 커널 크기, 1: 입력값 X 의 특성수, 32: 커널 수
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
# padding='SAME' : 커널 슬라이딩 시 이미지의 가장 외곽에서 한칸 더 밖으로 움직이는 옵션
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

# pooling layer
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [7]:
# second layer

# W1 [3, 3, 32, 64] -> [3 3]: 커널 크기, 32: 첫번째 컨볼루션 계층의 커널 수(출력층의 개수, 첫번째 컨볼루션 계층이 찾아낸 이미지의 특징 개수), 64: 커널 수
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
# 추출한 특징들 이용해 10개의 분류 만들어내는 계층

W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
# 7x7x64 크기의 1차원 계층으로 만든다
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [11]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob: 0.7})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료!')

#########
# 결과 확인
######
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                   Y: mnist.test.labels,
                                   keep_prob: 1}))

Epoch: 0001 Avg. cost = 0.356
Epoch: 0002 Avg. cost = 0.109
Epoch: 0003 Avg. cost = 0.079
Epoch: 0004 Avg. cost = 0.061
Epoch: 0005 Avg. cost = 0.051
Epoch: 0006 Avg. cost = 0.042
Epoch: 0007 Avg. cost = 0.035
Epoch: 0008 Avg. cost = 0.033
Epoch: 0009 Avg. cost = 0.029
Epoch: 0010 Avg. cost = 0.025
Epoch: 0011 Avg. cost = 0.021
Epoch: 0012 Avg. cost = 0.019
Epoch: 0013 Avg. cost = 0.017
Epoch: 0014 Avg. cost = 0.016
Epoch: 0015 Avg. cost = 0.015
최적화 완료!
정확도: 0.9894


**tf.layer 모듈 이용하여 코드 더 간단하게 만들기**

- 컨볼루션 및 풀링 계층의 코드

In [ ]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    ↓

In [ ]:
L1 = tf.layers.conv2d(X, 32, [3, 3], activation=tf.nn.relu)
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])

- 완전 연결 계층의 코드

In [ ]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)

    ↓

In [ ]:
L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)